In [ ]:
pip install transformers sentence-transformers gradio faiss-cpu pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import gradio as gr

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# Load the dataset
file_path = "train.csv"  # Replace with the actual dataset file path
data = pd.read_csv(file_path)

# Display the first few rows
print(data.head())

             qtype                                           Question  \
0   susceptibility  Who is at risk for Lymphocytic Choriomeningiti...   
1         symptoms  What are the symptoms of Lymphocytic Choriomen...   
2   susceptibility  Who is at risk for Lymphocytic Choriomeningiti...   
3  exams and tests  How to diagnose Lymphocytic Choriomeningitis (...   
4        treatment  What are the treatments for Lymphocytic Chorio...   

                                              Answer  
0  LCMV infections can occur after exposure to fr...  
1  LCMV is most commonly recognized as causing ne...  
2  Individuals of all ages who come into contact ...  
3  During the first phase of the disease, the mos...  
4  Aseptic meningitis, encephalitis, or meningoen...  


In [ ]:
# Preprocessing
data = data[['qtype', 'Question', 'Answer']].dropna()
data.columns = ['qtype', 'question', 'answer']  # Rename for simplicity

# Convert to lists
qtypes = data['qtype'].tolist()
questions = data['question'].tolist()
answers = data['answer'].tolist()

print(f"Dataset contains {len(questions)} questions across {data['qtype'].nunique()} categories.")

Dataset contains 16407 questions across 16 categories.


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Optimized for semantic search

In [ ]:
# Generate embeddings for all questions

question_embeddings = model.encode(questions, show_progress_bar=True)

# Save embeddings for future use
np.save("question_embeddings.npy", question_embeddings)
print("Embeddings created and saved.")

Batches:   0%|          | 0/513 [00:00<?, ?it/s]

Embeddings created and saved.


In [ ]:


# Initialize FAISS index
dimension = question_embeddings.shape[1]  # Embedding vector dimension
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the FAISS index
index.add(np.array(question_embeddings))
print(f"FAISS index contains {index.ntotal} items.")

FAISS index contains 16407 items.


In [ ]:
def search_faq(query, top_k=3):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)

    results = []
    for idx in indices[0]:
        results.append({
            'qtype': qtypes[idx],
            'question': questions[idx],
            'answer': answers[idx]
        })
    return results


In [ ]:
query = "What are the symptoms of flu?"
results = search_faq(query)

for result in results:
    print(f"Category: {result['qtype']}")
    print(f"Question: {result['question']}")
    print(f"Answer: {result['answer']}\n")

Category: symptoms
Question: What are the symptoms of Q Fever ?
Answer: Q fever can cause acute or chronic illness in humans, who usually acquire infection after contact with infected animals or exposure to contaminated environments. The acute symptoms caused by infection with Coxiella burnetii usually develop within 2-3 weeks of exposure, although as many as half of humans infected withC. burnetii do not show symptoms. 
 
The following is a list of symptoms commonly seen with acute Q fever. However, it is important to note that the combination of symptoms varies greatly from person to person. 
  
  - high fevers (up to 104-105°F)  
  - severe headache  
  - general malaise  
  - myalgia  
  - chills and/or sweats  
  - non-productive cough  
  - nausea  
  - vomiting  
  - diarrhea  
  - abdominal pain  
  - chest pain  
  
 
Although most persons with acute Q fever infection recover, others may experience serious illness with complications that may include pneumonia, granulomatous he

In [ ]:
def chatbot(query):
    results = search_faq(query)
    if results:
        top_result = results[0]
        response = (
            f"**Category:** {top_result['qtype']}\n"
            f"**Question:** {top_result['question']}\n"
            f"**Answer:** {top_result['answer']}"
        )
        return response
    else:
        return "Sorry, I couldn't find an answer to your question."


In [ ]:
import gradio as gr

interface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Enter your medical question..."),
    outputs="text",
    title="Healthcare Chatbot",
    description="Ask any medical question, and get instant answers with category-specific context!"
)


In [ ]:
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5aa42b8ef41ea0b4d6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


SyntaxError: invalid syntax (2049942660.py, line 1)

In [ ]:
def chatbot_with_category(query, category=None):
    results = search_faq(query)
    if category:
        # Filter results by category
        results = [res for res in results if res['qtype'].lower() == category.lower()]

    if results:
        top_result = results[0]
        response = (
            f"**Category:** {top_result['qtype']}\n"
            f"**Question:** {top_result['question']}\n"
            f"**Answer:** {top_result['answer']}"
        )
        return response
    else:
        return f"Sorry, no results found in the '{category}' category."

In [ ]:
interface = gr.Interface(
    fn=lambda query, category: chatbot_with_category(query, category),
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter your medical question..."),
        gr.Dropdown(choices=['symptoms', 'treatment', 'prevention', 'other'], label="Category (optional)")
    ],
    outputs="text",
    title="Healthcare Chatbot",
    description="Ask any medical question, and filter by category if desired!"
)

In [ ]:
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1712034a2820e6cb1b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
